# Note
**This notebook is only used for data cleaning on the initial data. This is a one time process only.**

In [1]:
# Convert from 60 Minutes Interval to 240 Minutes Interval
import pandas as pd
import plotly.graph_objects as go

df = pd.read_parquet("C:/Users/USER/projects/ai-agent-crypto-analyzer/tmp/20131006_20250331_XXBTZUSD_ohlc_hourly.parquet")

df["timestamp"] = pd.to_datetime(df['time'], unit='s')
df.index = df['timestamp']

start_date = df['timestamp'].min()
end_date = df['timestamp'].max()

print("Min Timestamp:", df['timestamp'].min())
print("Max Timestamp: ", df['timestamp'].max())

Min Timestamp: 2013-10-06 21:00:00
Max Timestamp:  2025-03-31 23:00:00


In [2]:
df.head()

,time,open,high,low,close,volume,count,timestamp
timestamp,,,,,,,,
2013-10-06 21:00:00,1381093200,122.00000,122.00000,122.00,122.00,0.1000,1,2013-10-06 21:00:00
2013-10-07 20:00:00,1381176000,123.61000,123.61000,123.61,123.61,0.1000,1,2013-10-07 20:00:00
2013-10-08 02:00:00,1381197600,123.91000,123.91000,123.90,123.90,1.9916,2,2013-10-08 02:00:00
2013-10-08 05:00:00,1381208400,124.19000,124.19000,124.18,124.18,2.0000,2,2013-10-08 05:00:00
2013-10-09 09:00:00,1381309200,124.01687,124.01687,123.84,123.84,2.8230,3,2013-10-09 09:00:00


In [3]:
df.tail(10)

,time,open,high,low,close,volume,count,timestamp
timestamp,,,,,,,,
2025-03-31 14:00:00,1743429600,82467.6,83710.0,82467.6,83469.7,204.750448,2685,2025-03-31 14:00:00
2025-03-31 15:00:00,1743433200,83494.9,83870.8,83399.9,83400.0,109.109145,3030,2025-03-31 15:00:00
2025-03-31 16:00:00,1743436800,83400.0,83688.4,83287.9,83289.1,86.640094,1962,2025-03-31 16:00:00
2025-03-31 17:00:00,1743440400,83289.2,83501.9,82592.4,82679.5,132.412958,2522,2025-03-31 17:00:00
2025-03-31 18:00:00,1743444000,82679.5,83509.3,82679.4,83303.7,112.499069,2133,2025-03-31 18:00:00
2025-03-31 19:00:00,1743447600,83286.1,83422.1,82368.9,82394.4,241.135396,3262,2025-03-31 19:00:00
2025-03-31 20:00:00,1743451200,82394.5,82798.7,82358.5,82427.7,93.071084,1617,2025-03-31 20:00:00
2025-03-31 21:00:00,1743454800,82427.7,82698.9,82427.7,82543.4,30.865337,939,2025-03-31 21:00:00
2025-03-31 22:00:00,1743458400,82543.7,82553.4,82259.3,82381.1,41.888085,888,2025-03-31 22:00:00


The intervals are inconsistent. The workaround will creating a new and complete interval of 1 hour and then use it as the new index in the exisisting dataframe. Then we will perform `forward fill` and `backward fill` in order.

In [4]:
full_timestamp = pd.date_range(start=start_date, end=end_date, freq='1h')
df = df.reindex(full_timestamp).ffill().bfill()

In [5]:
# Resample into 4-hour interval
df = df.resample("4h").agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum', 'count': 'sum'})

In [6]:
df = df.reset_index(names='timestamp')
df['time'] = df['timestamp'].astype('int64') // 10**9

In [7]:
df[['time','open','high','low','close','volume','count']].to_parquet(f"../tmp/{start_date.strftime('%Y%m%d')}_{end_date.strftime('%Y%m%d')}_XXBTZUSD_ohlc_240.parquet")